<a href="https://colab.research.google.com/github/Ana-PPS/data-and-analytcs/blob/MVP_II/MVP_b_SprintII_AnaPaulaSalgado_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PUC Rio: Pós Graduação em Ciência de Dados e Analytics**

**MVP Sprint II: Machine Learning & Analytics**
**Parte b**

Aluna: Ana Paula Pinheiro Salgado

Julho/2023

## Seção I: Introdução
**Classificador de imagens multiclasse - Animais**

**Contexto:** Temos um conjunto de imagens que representam 4 categorias de animais: vaca, galinha, cavalo e elefante. O objetivo deste notebook é criar um modelo de visão computacional que consiga classificar uma imagem dentre essas categorias.

**Estrutura:** O notebook encontra-se dividido da seguinte forma:

- Importação das bibliotecas
- Acesso e tratamento dos dados que serão a entrada do modelo de deep learning
- Configuração do modelo de deep learning usando uma rede neural convolucional simples com Keras
- Treinamento do modelo de deep learning
- Execução do modelo de deep learning treinado
- Avaliação do modelo de deep learning
- Exportação do modelo de deep learning
- Teste do modelo exportado

**Dataset:** A partir do dataset original, baixado do Kaggle (https://www.kaggle.com/datasets/alessiocorrado99/animals10), foram selecionadas apenas 4 pastas de imagens de forma a otimizar o tempo de execução do MVP.

### Importando as bibliotecas necessárias para executar o notebook
(Serão utilizadas as bibliotecas pandas e numpy, para a manipulação dos dados; matplotlib, para geração de gráficos; os, para manipulação de pastas e diretórios e bibliotecas voltadas para Machine e Deep Learning, tais como Keras, Tensor Flow e Scikit-Learn)


In [ ]:
# para usar o Google Drive
!pip install -q gdown
import gdown
from google.colab import drive

#para acessar e manipular arquivos, diretórios e estrutra de dados
import os
import shutil
from zipfile import ZipFile
import pandas as pd

# cálculos numéricos e operações matemáticas e trabalhar com números aleatórios
import numpy as np
import math
import random

# bibliotecas do keras para pré-processamento, modelos convolucionais e otimizadores dos modelos
!pip install -q tensorflow
!pip install -q keras
from tensorflow import keras
from keras.optimizers import Adam
from keras.preprocessing import image
#from keras.layers.experimental import preprocessing
from keras.layers import BatchNormalization
from keras import layers,models,Model
#from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras.preprocessing import image as keras_image
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf

# sckit-learn para pré-processamento e uso de métricas em machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
import sklearn.metrics as skm

# plotagem de gráficos, visualizações e imagens
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from skimage.io import imread
from skimage.transform import resize

# iterações
import itertools

# para uso de data e hora
from datetime import datetime

##Seção II: Acessando e tratando os dados que serão a entrada do modelo de deep learning

In [ ]:
# Baixando a pasta compactada para o notebook
file_id = "1bg_JhbuWv2lCCDyrfGI2yr5dROxbijYQ"

folder_path = f"https://drive.google.com/uc?id={file_id}"
output = "4animais.zip"
gdown.download(folder_path, output)

In [ ]:
# Descompactando o arquivo
with ZipFile('4animais.zip', 'r') as zip_object:
  zip_object.extractall()

In [ ]:
# Reunindo todos os arquivos em uma pasta para transformar em um dataset

path = "/content/"

classes = ['cavalo', 'elefante', 'galinha', 'vaca']

foldernames = os.listdir(path)

data = {"images": [], "animal": []}

for folder in classes:
    folderpath = os.path.join(path, folder)
    filelist = os.listdir(folderpath)
    for file in filelist:
        fpath = os.path.join(folderpath, file)
        data["images"].append(fpath)
        data["animal"].append(folder)


df = pd.DataFrame(data)

In [ ]:
print(df.head)
print(df.shape)

In [ ]:
#Resumo do dataset criado

print("Tamanho do dataset: ", df.shape)
print("_______________________________________")
print("Valores null: ")
print(df.isnull().sum())
print("_______________________________________")
print("Valores únicos: ")
print(df.nunique())

print("_______________________________________")
print("Qnt de imagens por categoria : ")
print(df.animal.value_counts())

print("_______________________________________")
print("Informação do dataset: ")
print(df.info())

In [ ]:
#Visualizando algumas imagens do conjunto de dados
fig = plt.gcf()
fig.set_size_inches(3*4, 3*4)
for i, row in df.sample(n=10).reset_index().iterrows():
    plt.subplot(2,5,i+1)
    image_path = row['images']
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(row["animal"])
    plt.axis('off')
plt.show()

In [ ]:
# Separação do dataframe em treino, teste e validação
df_train, Temp_df = train_test_split(df, train_size=0.7, random_state=13, shuffle=True)
df_test, df_val = train_test_split(Temp_df, test_size=0.6, random_state=30, shuffle=True)

In [ ]:
#Resumo da separação do conjunto de dados
print("#########Train##############")
print(df_train.head())
print(df_train.shape)
num_imagens_por_classe = df_train['animal'].value_counts()
print(num_imagens_por_classe)
print("#########Test###############")
print(df_test.head())
print(df_test.shape)
num_imagens_por_classe = df_test['animal'].value_counts()
print(num_imagens_por_classe)
print("#########Validação###############")
print(df_val.head())
print(df_val.shape)
num_imagens_por_classe = df_val['animal'].value_counts()
print(num_imagens_por_classe)

###  Preparação dos dados

No presente trabalho, será usado o método flow_from_dataframe() para gerar imagens aumentadas a partir de um dataframe, que aponta para as imagens originais. O método recebe parâmetros como o dataframe, o diretório com as imagens, o tamanho do lote (batch size) e o modo de classe (categorical, em função da quantidade de classes), entre outros.

O Data Augmentation aplica transformações aleatórias nas imagens existentes, o que ajuda a evitar overfitting e torna o modelo mais robusto, expondo-o a uma variedade maior de variações nas imagens.


In [ ]:
# Definindo o tamanho do batch, a dimensão das imagens e a quantidade de épocas
batch_size = 20
img_height = 224
img_width = 224
num_classes = 4
epochs = 15  # Para o MVP, utilizou-se o número pequeno para reduzir o tempo computacional

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)

test_datagen=ImageDataGenerator(rescale=1./255)


train_generator=train_datagen.flow_from_dataframe(
    dataframe=df_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='images',
    y_col='animal',
    color_mode ='rgb',
    seed = 13,
    shuffle=False
    )

val_generator=val_datagen.flow_from_dataframe(
    dataframe=df_val,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='images',
    y_col='animal',
    color_mode ='rgb',
    shuffle=False
    )

test_generator=test_datagen.flow_from_dataframe(
    dataframe = df_test,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    x_col='images',
    y_col='animal',
    color_mode ='rgb',
    shuffle=False)

## Seção III: Configuração do Modelo de Deep Learning

### Configuração de um modelo de deep learning usando uma rede neural convolucional (CNN) simples com a biblioteca Keras

O modelo será construído com 4 camadas convolucionais 2D, com 32, 64, 128 e 256 filtros e com função de ativação `ReLU`. Inseridas também 2 camadas de Dropout para tentar melhorar a generalização e reduzir overfitting. Na sequência é adicionada uma camada `softmax` com a mesma função de ativação.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_height,img_width,3)),
    keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),  # Aumentar a complexidade com mais unidades
    keras.layers.Dropout(0.2),  # Incluir mais uma camada de dropout
    keras.layers.Dense(128, activation='relu'),  # Adicionar outra camada Dense
    keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# Resumindo o modelo que será utilizado
model.summary()

### Treinamento com o modelo otimizado

Utilizado o modelo Adam para otimização, perda do tipo de entropia cruzada, por sem um problema de classificação multiclasse e métrica de acurácia para avaliação do modelo.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=len(val_generator),
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5,
                                             restore_best_weights=True)])


#### Visualização de métricas da avaliação do modelo treinado

A entropia cruzada mede a diferença entre duas distribuições de probabilidade (verdadeira e falsa) sobre um mesmo conjunto de dados.

A acurácia é a proximidade de um resultado predito com o seu valor de referência real.

Conforme se aumenta o número de épocas, espera-se que o parâmetro loss diminua (para o dataset de treino e também para o de validação) e que a acurácia aumente.

Além disso, é possível inferir sobre under ou overfitting do modelo a depender dos resultados para os conjuntos de treino e validação.

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, val_accuracy, label = 'Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs Number')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epochs Number')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Seção IV: Execução do modelo de deep learning treinado nas imagens de teste

#### Execução do modelo com imagens do dataframe de teste



In [ ]:
count_images = 0
class_names = ['cavalo',
               'elefante',
               'galinha',
               'vaca']
y_pred = list() # para armazenar as categorias preditas das imagens do dataset de teste
y_true = list() # para armazenar as categorias reais das imagens do dataset de teste

In [ ]:
# Reimportando a biblioteca treino, pois, durante os testes com o notebook, esta célula apresentava erro
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Percorrendo as imagens do dataframe de teste para aplicação de modelo treinado
for img_path, label in zip(df_test['images'], df_test['animal']):
     _, file_extension = os.path.splitext(img_path)
     if file_extension.lower() in ['.jpeg', '.jpg', '.png']:
         count_images += 1

         split_path = img_path.split('/')
         label = split_path[2]
         y_true.append(label)

         img = Image.open(img_path).resize((300, 300))
         display(img)

         img = load_img(img_path, target_size=(img_height, img_width))
         x = img_to_array(img)
         x = np.expand_dims(x, axis=0)
         x = x.astype('float32') / 255.0

         # Predições
         prediction = model.predict(x)

         # Printando as saídas do modelo
         predicted_class = np.argmax(prediction[0])
         probability = prediction[0][predicted_class]
         y_pred.append(class_names[predicted_class])
         print("Categoria Real: ", label)
         print("Categoria Predita: ", class_names[predicted_class])
         print("Probabilidade: ", probability)
         print("\n")

In [ ]:
# Calculando as métricas do modelo aplicado as imagens de teste
accuracy = skm.accuracy_score(y_true, y_pred)
precision = skm.precision_score(y_true, y_pred, average='weighted')
recall = skm.recall_score(y_true, y_pred, average='weighted')
f1score = skm.f1_score(y_true, y_pred, average='weighted')

# Arredondar os valores para duas casas decimais
accuracy = round(accuracy, 2)
precision = round(precision, 2)
recall = round(recall, 2)
f1score = round(f1score, 2)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1score)


In [ ]:
classification_rep = classification_report(y_true, y_pred, target_names=class_names)
print("\nRelatório de Classificação:")
print(classification_rep)

O modelo apresentou uma acurácia de 79% no geral, o que significa que a maioria dos exemplos do conjunto de dados foram classificados corretamente.
A classe 'vaca' apresenta um índice de recall bem inferior aos demais, o que denota dificuldade do modelo em identificar esta classe.

A Matriz de Confusão oferece um detalhamento do desempenho do modelo, mostrando, para cada classe, o número de classificações corretas em relação ao número de classificações indicadas pelo modelo

In [ ]:
y_pred = np.array(y_pred)
y_true = np.array(y_true)

cm = confusion_matrix(y_true, y_pred)

# Definindo função para plotar a matriz de confusão
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print('Matriz de confusão sem normalização')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt_str = '{:d}' if not normalize else '{:.2f}'

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, fmt_str.format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Label real')
    plt.xlabel('Label predito')

    plt.figure(figsize=(12,12))
plot_confusion_matrix(cm, classes, normalize=False, title='Matriz de Confusão')
plt.show()


#### Execução do modelo treinado com test generator (Essa parte foi utilizada pois o código de teste da sessão anterior não estava funcionando adequadamente, ficando apenas de histórico neste notebook)

In [ ]:
# y_true_tg = test_generator.classes
# y_pred_tg = np.argmax(model.predict(test_generator), axis=1)

# classes = dict(zip(test_generator.class_indices.values(), test_generator.class_indices.keys()))
# Predictions = pd.DataFrame({"Image Index": list(range(len(test_generator.labels))), # Índice da imagem no conjunto de teste
#                             "Test Labels": test_generator.labels, # Rótulos reais das imagens no conjunto de teste
#                             "Test Classes": [classes[i] for i in test_generator.labels], # Classes correspondentes aos rótulos reais
#                             "Prediction Labels": y_pred_tg, # Rótulos previstos pelo modelo
#                             "Prediction Classes": [classes[i] for i in y_pred_tg], # Classes correspondentes aos rótulos previstos
#                             "Path": test_generator.filepaths, #Caminho dos arquivos de imagem
#                             "Prediction Probability": [x for x in np.asarray(np.max(model.predict(test_generator), axis=1))] #Probabilidade da classe prevista pelo modelo
#                            })
# Predictions.head(10)

Plotando um subconjunto das imagens de teste com as respectivas previsões

In [ ]:
# Obtendo uma amostra aleatória de 10 linhas do DataFrame Predict

# sample_predictions = Predictions.sample(n=10)

# for index, row in sample_predictions.iterrows():
#     # Obtendo as informações necessárias para visualização e impressão
#     image = plt.imread(row['Path'])
#     true_label = row['Test Classes']
#     predicted_label = row['Prediction Classes']
#     probability = row['Prediction Probability']

#     # Arredondando a probabilidade para duas casas decimais
#     probability = round(probability, 2)

#     # Configuração da figura para exibir a imagem
#     plt.figure()
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()

#     # Impressão dos resultados da classificação
#     print("True Label: ", true_label)
#     print("Predicted Label: ", predicted_label)
#     print("Probability: ", probability)
#     print()

In [ ]:
# results = model.evaluate(test_generator, verbose=0)

# print("    Test Loss: {:.2f}".format(results[0]))
# print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
# Calculando as métricas do modelo aplicado as imagens do teste generator
# accuracy = skm.accuracy_score(y_true_tg, y_pred_tg)
# precision = skm.precision_score(y_true_tg, y_pred_tg, average='weighted')
# recall = skm.recall_score(y_true_tg, y_pred_tg, average='weighted')
# f1score = skm.f1_score(y_true_tg, y_pred_tg, average='weighted')

# # Arredondar os valores para duas casas decimais
# accuracy = round(accuracy, 2)
# precision = round(precision, 2)
# recall = round(recall, 2)
# f1score = round(f1score, 2)

# print("Accuracy: ", accuracy)
# print("Precision: ", precision)
# print("Recall: ", recall)
# print("F1 Score: ", f1score)
# print(classification_report(y_true_tg, y_pred_tg, target_names=test_generator.class_indices.keys()))

In [ ]:
# Definindo função para plotar a matriz de confusão
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     Esta função imprime e plota a matriz de confusão.
#     A normalização pode ser aplicada definindo `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Matriz de confusão normalizada")
#     else:
#         print('Matriz de confusão sem normalização')

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt_str = '{:d}' if not normalize else '{:.2f}'

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, fmt_str.format(cm[i, j]),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('Label real')
#     plt.xlabel('Label predito')

# class_names = list(test_generator.class_indices.values())
# y_pred_tg = Predictions['Prediction Labels']
# y_test_tg = Predictions['Test Labels']
# Map_class = test_generator.class_indices

# cnf_matrix = confusion_matrix(y_true_tg, y_pred_tg, labels=class_names)
# np.set_printoptions(precision=2)

# plt.figure()
# plot_confusion_matrix(cnf_matrix,
#                       classes=Map_class,
#                       normalize=False,
#                       title='Matriz real x predição')

## Seção V: Exportação do modelo de deep learning para posterior uso

Salvando o modelo de deep learning que foi treinado

In [ ]:
# obtendo a data e hora atual
now = datetime.now()

# Definição do formato
format = '%Y-%m-%dT%H%M'

# Converter a data e hora em uma string com o formato especificado
formatted_datetime = now.strftime(format)

path_model = 'content/trained_models'

name_model = 'trained_model_' + formatted_datetime + '.h5'

# salvando o modelo
model.save("%s/%s" % (path_model, name_model))
print("Modelo salvo com o nome: ", name_model)

## Seção VI: Teste do modelo exportado

Carregando o modelo salvo

In [ ]:
loaded_model = keras.models.load_model("%s/%s" % (path_model, name_model))
print("Modelo %s carregado com sucesso" % (name_model))

Executando o modelo exportado para acompanhar as classificações de cada uma das imagens de teste

In [ ]:
# Importando a pasta com as novas fotos
from google.colab import drive

# Montar o Google Drive
drive.mount('content/drive')

parent_dir = 'content/drive/MyDrive/dataset/usp=drive_link'
count = 0
class_names = ['cavalo', 'elefante', 'galinha', 'vaca']

y_pred = list()
y_true - list()

for subdir, dirs, files in os.walk(parent_dir):

    for file in files:

        if file.endswith('.png') or file.endswith('.jpg'):

            count_images+=1
            split_path = os.path.join(subdir, file).split('/')
            label = split_path[-2]
            y_true.append(label)

            img_path = os.path.join(subdir, file)
            display(ipimg(filename=img_path, width=300))

            img = image.load_img(img_path, target_size=(img_height, img_width))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x.astype('float32') / 255.0

            # Previsão
            prediction = loaded_model.predict(x)

            # Printando as saídas do modelo
            predicted_class = np.argmax(prediction[0])
            probability = prediction[0][predicted_class]
            y_pred.append(class_names[predicted_class])
            print("Label:", label)
            print("Previsão:", class_names[predicted_class])
            print("Probabilidade:", probability)
            print("\n")
